In [2]:
import pandas as pd
import os
from datetime import datetime

In [6]:

# Load the existing merged dataframe
merged_df = pd.read_csv('/Users/gadimg/Library/Mobile Documents/com~apple~CloudDocs/PhD dis/third/Data/Merged_data/Full_with_monthly_no_other_crypto.csv', parse_dates=['Date'], dayfirst=True)

# Directory containing the new CSV files
directory = '/Users/gadimg/Library/Mobile Documents/com~apple~CloudDocs/PhD dis/third/Data/Other_major_crypto'

# Loop through each CSV file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Load the current CSV
        df = pd.read_csv(os.path.join(directory, filename), parse_dates=['Date'], dayfirst=True)
        
        # Filter out dates beyond the specified range
        start_date = pd.to_datetime('20/08/2020', dayfirst=True)
        end_date = pd.to_datetime('20/10/2023', dayfirst=True)
        df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]


        # Make sure the dataframe has no missing dates
        all_dates = pd.date_range(start=start_date, end=end_date, freq='D')
        df = df.set_index('Date').reindex(all_dates).reset_index().rename(columns={'index': 'Date'})
        
        # Interpolate missing values
        df.interpolate(method='linear', inplace=True)
        
        # Take 'Close' and 'Volume' columns and rename them to avoid column name clashes
        close_col = f"{filename[:-4]}_Close"  # Assuming filenames don't have dots except for '.csv'
        volume_col = f"{filename[:-4]}_Volume"
        df = df.rename(columns={'Close': close_col, 'Volume': volume_col})
        
        # Drop other columns
        df = df[['Date', close_col, volume_col]]
        
        # Merge with the main dataframe
        merged_df = pd.merge(merged_df, df, on='Date', how='left')

# Save the updated merged dataframe
merged_df.to_csv('/Users/gadimg/Library/Mobile Documents/com~apple~CloudDocs/PhD dis/third/Data/Merged_data/updated_merged.csv', index=False)
